In [1]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api import TV
from tmdbv3api import Genre, Search

tmdb = TMDb()
tmdb.api_key = '6727d8b68d506e9160ce54956c29d855'
tmdb.language = 'en'
tmdb.debug = True

movie = Movie()
tv = TV()
genre = Genre()

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import json as json
import pickle

In [4]:
stc = pd.read_csv('/Users/dinaabed/Desktop/Final_Dataset.txt', delimiter = ",")
stc = stc.drop('Unnamed: 0', axis=1)
df = stc.copy() # the data is heavy to reread so this is our check poing

# creating unique id for videos
df['vid_id'] = df.groupby(['original_name']).ngroup()
df.head()

,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,hd,original_name,vid_id
0,2017-05-27,26138,100 treets,40,MOVIE,0,0,Drama Movie100 Streets,Drama,0,0,100 treets,2
1,2017-05-21,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,1,Moana,924
2,2017-08-10,7418,The Mermaid Princess,8,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess,1524
3,2017-07-26,19307,The Mermaid Princess,76,MOVIE,0,0,Animation MovieThe Mermaid Princess (HD),Animation,0,1,The Mermaid Princess,1524
4,2017-07-07,15860,Churchill,87,MOVIE,0,0,Biography MovieChurchill (HD),Biography,0,1,Churchill,317


In [5]:
df['original_name'] = [" ".join(x.split()) for x in df['original_name']]

# get the names of TVs and movies
movie_orig = list(set(df[df['program_class'] == 'MOVIE']['original_name']))
tv_orig = list(set(df[df['program_class'] != 'MOVIE']['original_name']))



Number of TV Shows 276
Number of Movies 1525


In [60]:
#number of movies
format(len(movie_orig))

'1525'

In [61]:
#number of shows
format(len(tv_orig))

'276'

In [44]:
def get_tmdb_id(names, types):

    item_api_id = []
    item_name_not_found = []
    item_dic = []
    for name in names:
        search  = types.search(name) # search api for names
        
        if len(search) != 0: # if nothing, meant we do not have the name in tmdb
            search = search[0]
            item_dic.append(search) # add found items in list
            item_api_id.append(search.id)
        else:
            item_name_not_found.append(name)
    # return 3 
    return item_api_id, item_name_not_found, item_dic

In [28]:
movie_api_id, movie_name_not_found, movie_dic = get_tmdb_id(movie_orig, movie)
tv_api_id, tv_name_not_found, tv_dic = get_tmdb_id(tv_orig, tv)

In [29]:
print(movie_dic[0])


{'adult': False, 'backdrop_path': '/3Skg7e94WFreLgQAR0PAYSk0Zv1.jpg', 'genre_ids': [10751, 12, 16, 18], 'id': 37135, 'original_language': 'en', 'original_title': 'Tarzan', 'overview': "Tarzan was a small orphan who was raised by an ape named Kala since he was a child. He believed that this was his family, but on an expedition Jane Porter is rescued by Tarzan. He then finds out that he's human. Now Tarzan must make the decision as to which family he should belong to...", 'popularity': 100.752, 'poster_path': '/a0Syo09bmNJgehjfeVL6meR9Mjh.jpg', 'release_date': '1999-06-18', 'title': 'Tarzan', 'video': False, 'vote_average': 7.4, 'vote_count': 5741}


In [73]:

# get all the genres ids list from tmdb api bcz all i get is genre ID
genres_tv = genre.tv_list()
genres_movie = genre.movie_list()
genres_tv.extend(genres_movie)
all_genres = genres_tv



In [45]:
#numer Shows in the dataset 
format(len(tv_orig))

'276'

In [48]:
#number of movies in the dataset 
format(len(movie_orig))

'1525'

In [49]:
#number of shows not found in TMDB

format(len(tv_name_not_found))

'141'

In [51]:
#number of shows not found in TMDB

format(len(movie_name_not_found))


'192'

____________________________

# iwant to save the dictionary of movies and tv shows in to csv

In [74]:
def exp_dic(dic, filename):
    import csv
    csv_columns = list(dic[0].keys())
    dict_data = dic
    csv_file = "{}.csv".format(filename)
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in dict_data:
                writer.writerow(data)
    except IOError:
        print("error")

In [75]:
exp_dic(tv_dic, 'tv_df')
exp_dic(movie_dic, 'movie_df')
exp_dic(all_genres, 'genre_df')

In [76]:
# loading the dataframes
movie_df  = pd.read_csv('movie_df.csv')
tv_df     = pd.read_csv('tv_df.csv')
genre_df  = pd.read_csv('genre_df.csv')
genre_df  = genre_df.drop_duplicates()

# adding program type 
movie_df['program_type'] = 'Movie'
tv_df['program_type'] = 'TV Show'

In [77]:
def change_string_list(df, column):
   
    print('Before: {}, type: {}'.format(df[column][1], type(df[column][1])))
    
    for i in range(len(df[column])):
        df[column][i] = json.loads(df[column][i])
    
    print('After: {}, type: {}'.format(df[column][1], type(df[column][1])))

In [78]:
change_string_list(movie_df, 'genre_ids')
change_string_list(tv_df, 'genre_ids')


Before: [27, 18, 9648], type: <class 'str'>
After: [27, 18, 9648], type: <class 'list'>
Before: [18, 10765], type: <class 'str'>
After: [18, 10765], type: <class 'list'>


In [79]:
def Id_into_genre(df):
    print('Before: {}'.format(df['genre_ids'][1]))

    for i in range(len(df)):
        lists = df['genre_ids'][i]
        names = list(genre_df[genre_df['id'].isin(lists)]['name'])
        df.loc[:,'genre_ids'][i] = names
          
    print('After: {}'.format(df['genre_ids'][1]))

In [80]:
Id_into_genre(tv_df)
Id_into_genre(movie_df)

Before: [18, 10765]
After: ['Drama', 'Sci-Fi & Fantasy']
Before: [27, 18, 9648]
After: ['Drama', 'Mystery', 'Horror']


In [71]:
tv_df.head()


,backdrop_path,first_air_date,genre_ids,id,name,origin_country,original_language,original_name,overview,popularity,poster_path,vote_average,vote_count,program_type
0,/jdri5Eu0yQfPe6h0MbOM7hbXrZC.jpg,2012-09-28,"[Action & Adventure, Animation, Kids, Sci-Fi &...",51817,Teenage Mutant Ninja Turtles,['US'],en,Teenage Mutant Ninja Turtles,The Teenage Mutant Ninja Turtles are back in a...,61.216,/n0DN6kJf0kGeuNFUYLT6eZe5ERR.jpg,8.3,483,TV Show
1,/MaQ7hbNsiJ30p14UgRdEnXDGMH.jpg,2015-01-06,"[Drama, Sci-Fi & Fantasy]",61550,Marvel's Agent Carter,['US'],en,Marvel's Agent Carter,"It's 1946, and peace has dealt Peggy Carter a ...",56.272,/8Y30TnBtGT4C7RI3nruPaIGGSkZ.jpg,7.6,1436,TV Show
2,/tLwEXm2nG64Tce00d59rPNI8ePh.jpg,2013-01-30,"[Crime, Drama]",46533,The Americans,['US'],en,The Americans,"Set during the Cold War period in the 1980s, T...",44.429,/lD4gTLrI81Crs6CDwwjQrnkJrUm.jpg,7.9,657,TV Show
3,/rUJ49zLaoBVjsxfRUiCsXNdOYBW.jpg,2014-10-13,"[Animation, Family, Kids]",64198,Blaze and the Monster Machines,['US'],en,Blaze and the Monster Machines,AJ is an 8-year-old techie who drives monster-...,43.773,/vDTuk6TxOzjzHzrcOw0w27nwHa6.jpg,6.2,18,TV Show
4,/rMbHGg9J5jnhuIX0Lkv4r7yZ7QI.jpg,1999-08-30,"[Action & Adventure, Animation]",20992,Sonic Underground,"['FR', 'US']",en,Sonic Underground,"Sonic, Sonia, and Manic are the children of Qu...",12.648,/rVjYevHn5RxkIkk4SwNnqIS8LJB.jpg,6.9,123,TV Show


In [72]:
movie_df

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,program_type
0,False,/3Skg7e94WFreLgQAR0PAYSk0Zv1.jpg,"[Animation, Drama, Family, Adventure]",37135,en,Tarzan,Tarzan was a small orphan who was raised by an...,100.752,/a0Syo09bmNJgehjfeVL6meR9Mjh.jpg,1999-06-18,Tarzan,False,7.4,5741,Movie
1,False,/rHUKzMqqsrXQjRO8OobnKy2qZ4Z.jpg,"[Drama, Mystery, Horror]",287424,en,Maggie,There's a deadly zombie epidemic threatening h...,28.695,/twa9gXjocLM629sJrQYVtmF306R.jpg,2015-05-08,Maggie,False,5.4,1403,Movie
2,False,/7T1o5UoMtw74So1EGoQMLndIeWz.jpg,[Horror],14254,en,The Uninvited,Anna returns home after spending time in a psy...,25.290,/7Aw5L42HcyUTEeKNNRqGtoBI7sX.jpg,2009-01-30,The Uninvited,False,6.5,1236,Movie
3,False,/v63E5H5TD7Eay1gZPVt9ibP9riy.jpg,"[Action, Fantasy, Horror]",52520,en,Underworld: Awakening,"Having escaped years of imprisonment, vampire ...",73.545,/jN0uuc8U6M3sTg9zEaliJV60Stf.jpg,2012-01-19,Underworld: Awakening,False,6.3,3548,Movie
4,False,/wxkXCVGw61kbJAfP9SkxoETD0uz.jpg,"[Animation, Comedy, Family, Adventure, Fantasy]",810,en,Shrek the Third,The King of Far Far Away has died and Shrek an...,114.508,/jaNe16gE9zdAYyVwRqmlmRKshHm.jpg,2007-05-17,Shrek the Third,False,6.3,7542,Movie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1328,False,/f4lMXJZgCAlxazlERC3TBqc68vs.jpg,[Drama],291351,en,The Sea of Trees,"Numbed by grief, Arthur Brennan heads across t...",8.982,/gmVGIEfzxupZUTer1Z5n80CHFnD.jpg,2016-04-27,The Sea of Trees,False,6.3,417,Movie
1329,False,/4IblUyXb3ZuPuMbGB0Ywcd2dFVG.jpg,"[Drama, Mystery, Action, Thriller]",8834,en,Conspiracy Theory,A man obsessed with conspiracy theories become...,20.784,/wj0n6gci4EMWRhV9ozCKTgESGdB.jpg,1997-08-08,Conspiracy Theory,False,6.6,1172,Movie
1330,False,/5HnHEodzdN7thQRMfncWdle8ybD.jpg,"[Comedy, Family, Fantasy]",322240,en,Nine Lives,A stuffy businessman finds himself trapped ins...,25.509,/i457G22lL9t2yhEFdYz5Nd1M5DV.jpg,2016-08-03,Nine Lives,False,5.8,910,Movie
1331,False,/rdDcZHklNkHHApjcBd0qTK7YzJr.jpg,"[Action, Adventure]",308504,en,Last Knights,"When an evil emperor executes their leader, hi...",22.512,/uGqlk7Hu8FRLVDcFZ16NxTy09ZE.jpg,2015-04-03,Last Knights,False,6.5,921,Movie


In [81]:
# change the names of the columns so that it is the same as movie df 
tv_df.columns = ['backdrop_path', 'release_date', 'genre_ids', 'id', 'name',
       'origin_country', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'vote_average', 'vote_count',
       'program_type']

column_needed = ['id', 'original_title','release_date','program_type', 
                 'genre_ids', 'overview', 'popularity', 'vote_average', 'vote_count']

movie_df = movie_df[column_needed]
tv_df = tv_df[column_needed]

dfmerged = [movie_df, tv_df]
df  = pd.concat(dfmerged).reset_index()
df  = df.drop('index', axis=1)

df.columns = ['movie_id', 'original_name','release_date',
              'program_type', 'genres', 'overview', 
              'popularity', 'vote_average', 'vote_count']

# removing  from genres
df['genres'] = [", ".join(x) for x in df['genres']]
# the release date
df['release_date'] = [str(x)[:4] for x in df['release_date']]

df.head(3)


,movie_id,original_name,release_date,program_type,genres,overview,popularity,vote_average,vote_count
0,37135,Tarzan,1999,Movie,"Animation, Drama, Family, Adventure",Tarzan was a small orphan who was raised by an...,100.752,7.4,5741
1,287424,Maggie,2015,Movie,"Drama, Mystery, Horror",There's a deadly zombie epidemic threatening h...,28.695,5.4,1403
2,14254,The Uninvited,2009,Movie,Horror,Anna returns home after spending time in a psy...,25.290,6.5,1236


fetching what i found in the API and matching it with the main dataset

In [82]:
#  only stc df to match found movies full information
new = list(df['original_name'].values)
# made a new  dataframe
stclast = stc[stc['original_name'].isin(new)]
stc_working = stc[~stc['original_name'].isin(new)]

In [84]:
# concat columns for better info about the content
complete_df = pd.merge(stclast, df, how='left', indicator=True)
complete_df.head()

,date_,user_id_maped,program_name,duration_seconds,program_class,season,episode,program_desc,program_genre,series_title,...,original_name,movie_id,release_date,program_type,genres,overview,popularity,vote_average,vote_count,_merge
0,2017-05-21,7946,Moana,17,MOVIE,0,0,Animation MovieMoana (HD),Animation,0,...,Moana,277834,2016,Movie,"Animation, Comedy, Family, Adventure","In Ancient Polynesia, when a terrible curse in...",47.750,7.6,10631,both
1,2017-07-07,15860,Churchill,87,MOVIE,0,0,Biography MovieChurchill (HD),Biography,0,...,Churchill,399790,2017,Movie,"Drama, History",A ticking-clock thriller following Winston Chu...,15.256,6.1,260,both
2,2018-03-29,6358,Coco,14,MOVIE,0,0,Animation MovieCoco (HD),Animation,0,...,Coco,354912,2017,Movie,"Animation, Comedy, Family, Adventure, Fantasy,...",Despite his family’s baffling generations-old ...,234.074,8.2,16037,both
3,2018-01-27,11660,Kidnap,85,MOVIE,0,0,Action MovieKidnap (HD),Action,0,...,Kidnap,293768,2017,Movie,"Drama, Thriller",A mother (in her Minivan) stops at nothing to ...,35.231,6.2,1130,both
4,2017-03-30,5155,The Accountant,42,MOVIE,0,0,Action MovieThe Accountant (HD),Action,0,...,The Accountant,302946,2016,Movie,"Crime, Drama, Thriller",As a math savant uncooks the books for a new c...,35.596,7.0,5158,both


In [86]:
def processing_dataframe(df):

    df.columns = ['stream_date', 'user_id', 'program_name', 'duration_seconds',
       'program_class', 'season', 'episode', 'program_desc', 'program_genre',
       'series_title', 'hd', 'original_name', 'program_id', 'release_date',
        'program_type', 'genres', 'overview', 'popularity', 'vote_average',
        'vote_count', 'merge']
    
    df['total_views'] = df.groupby('original_name', as_index=False)['user_id'].transform(lambda x: x.count())
    # feature: for content, ['overview', 'genres', 'release_date']
    df['genres'] = [x.replace(',', '') for x in df['genres']]

    df['content'] = df['genres']+  df['release_date'] + df['overview']
    
    df = df.drop_duplicates(subset = 'original_name').reset_index()
    df = df.drop('index', axis=1)

    df = df[['original_name', 'release_date', 'genres', 'overview','total_views', 'popularity', 'vote_average', 'vote_count', 'content']]
    return df

In [87]:
recomendation_df = processing_dataframe(complete_df)


In [92]:
recomendation_df.sample(5)

,original_name,release_date,genres,overview,total_views,popularity,vote_average,vote_count,content
89,Barnyard,2006,Animation Comedy Family,"When the farmer's away, all the animals play, ...",934,21.873,5.8,880,Animation Comedy Family2006When the farmer's a...
830,Lethal Weapon,1987,Comedy Crime Action Adventure Thriller,Veteran buttoned-down LAPD detective Roger Mur...,10,37.500,7.3,3743,Comedy Crime Action Adventure Thriller1987Vete...
412,Up,2009,Animation Comedy Family Adventure,Carl Fredricksen spent his entire life dreamin...,89,131.869,7.9,17490,Animation Comedy Family Adventure2009Carl Fred...
627,The Golden Compass,2007,Adventure Fantasy,"After overhearing a shocking secret, precociou...",126,25.374,6.0,3957,Adventure Fantasy2007After overhearing a shock...
553,It Follows,2014,Mystery Horror,When carefree teenager Jay sleeps with her old...,545,56.349,6.6,5391,Mystery Horror2014When carefree teenager Jay s...


In [93]:
recomendation_df.to_csv("compstc.csv")


In [ ]:
# feature extracting


